In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 读入图片并显示

In [ ]:
file = r"D:\Fenkx\Fenkx - General\AI\Dataset\BarCode\My Datasets\Factory\772FBFU0MPZZZS02H0_NG_BarCode_Camera3_1205171258.jpg"
image_source = cv2.imdecode(np.fromfile(file, dtype=np.uint8), 1)
plt.figure(dpi=150)
plt.subplot(2, 1, 1)
plt.xticks([])
plt.yticks([])
plt.imshow(cv2.cvtColor(image_source, cv2.COLOR_BGR2RGB))
plt.subplot(2, 1, 2)
plt.hist(image_source.ravel(), 256, [0, 256], color='r')
plt.xlim([0, 256])
plt.xlabel('Gray Level')
plt.ylabel('Number of Pixels')
plt.title('Original Histogram')
plt.show()

# 掩模锐化

In [ ]:
# 高斯模糊
blurred = cv2.GaussianBlur(image_source, (5, 5), 0, cv2.CV_64F)
res = image_source.astype(np.float64) - blurred

# 增强后的图像
enhanced_image = image_source + res
enhanced_image = np.clip(enhanced_image, 0, 255).astype(np.uint8)
# cv2.imwrite(r"C:\Users\vance\Desktop\1.png", enhanced_image)
plt.figure(dpi=300)
plt.imshow(cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB))
plt.show()

# 图像金字塔

In [ ]:
from tools.ImageOperate import pyrdown_multithread

image_pydown = pyrdown_multithread(enhanced_image)
image_pydown_s = image_pydown[-1]

print(image_pydown_s.shape)

plt.figure(dpi=300)
plt.imshow(cv2.cvtColor(image_pydown_s, cv2.COLOR_BGR2RGB))
plt.show()
# plt.subplot(2, 1, 1)
# plt.xticks([])
# plt.yticks([])
# plt.imshow(image_pydown_s)
#
# plt.subplot(2, 1, 2)
# plt.hist(image_pydown_s.ravel(), 256, [0, 256], color='r')
# plt.xlim([0, 256])
# plt.xlabel('Gray Level')
# plt.ylabel('Number of Pixels')
# plt.title('Pydown Histogram')
# plt.show()

In [ ]:
# 高斯模糊
blurred1 = cv2.GaussianBlur(image_pydown_s, (5, 5), 0, cv2.CV_64F)
res1 = image_pydown_s.astype(np.float64) - blurred1

# 增强后的图像
image_pydown_s = image_pydown_s + res1
image_pydown_s = np.clip(image_pydown_s, 0, 255).astype(np.uint8)
plt.figure(dpi=300)
plt.imshow(cv2.cvtColor(image_pydown_s, cv2.COLOR_BGR2RGB))
plt.show()

# 去除异常亮块

## 阈值分割

In [ ]:
image_gray = cv2.cvtColor(image_pydown_s, cv2.COLOR_BGR2GRAY)
ret, image_threshold = cv2.threshold(image_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
plt.imshow(image_threshold, 'gray')
plt.show()

## 直线检测

In [ ]:
area_threshold = int(image_gray.shape[0] * image_gray.shape[1] * 0.015)
print("area_threshold:{}".format(area_threshold))

contours1, hierarchy1 = cv2.findContours(image_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

lsd = cv2.createLineSegmentDetector()
lines_list1 = []

for contour1 in contours1:
	x, y, w, h = cv2.boundingRect(contour1)
	if cv2.contourArea(contour1) > area_threshold:
		image_gray_part = image_gray[y:y + h, x:x + w]
		ret1, image_threshold1 = cv2.threshold(image_gray_part, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)
		# detect lines1
		lines1, width, prec, nfa = lsd.detect(image_threshold1)
		if lines1 is not None:
			if lines1.shape[0] < 70:
				cv2.drawContours(image_gray, [contour1], -1, 0, cv2.FILLED)
			else:
				pass
			lines_list1.append(lines1)
	else:
		cv2.drawContours(image_gray, [contour1], -1, 0, cv2.FILLED)
# image_draw = cv2.minAreaRect([contours1],(0, 0, 255))
plt.xticks([])
plt.yticks([])
plt.imshow(image_gray, 'gray')
plt.show()

# 灰度重映射

In [ ]:
imax = np.max(image_gray)
image_equalized = (255 ** (image_gray / imax)).astype(np.uint8)
plt.figure(dpi=200)
plt.subplot(2, 2, 1)
plt.hist(image_gray.ravel(), 256, [0, 256], color='r')
plt.subplot(2, 2, 2)
plt.hist(image_equalized.ravel(), 256, [0, 256], color='r')

plt.subplot(2, 2, 3)
plt.imshow(image_gray, 'gray')
plt.subplot(2, 2, 4)
plt.imshow(image_equalized, 'gray')
plt.show()

# 条码区域定位

In [ ]:
r, t = cv2.threshold(image_equalized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
plt.figure(dpi=300)
plt.imshow(t, 'gray')
plt.show()

In [ ]:
from LineCluster import find_barcode_by_cluster, draw_clusters, cluster_lines

clusters = find_barcode_by_cluster(t, 0.5)
# lsd = cv2.createLineSegmentDetector()
# lines, width, prec, nfa = lsd.detect(t)
# lines =np.reshape(lines,(-1,4))
# clusters = cluster_lines(lines, eps=1)

image_drawed = draw_clusters(image_pydown_s, clusters)

plt.figure(dpi=300)
plt.imshow(cv2.cvtColor(image_drawed, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
kernel = np.ones((3, 3), np.uint8)
closing = cv2.morphologyEx(t, cv2.MORPH_OPEN, kernel)  #闭运算
closing = cv2.morphologyEx(t, cv2.MORPH_CLOSE, kernel)  #闭运算

contours1, hierarchy1 = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

lines_list2 = []
for contour2 in contours1:
	x, y, w, h = cv2.boundingRect(contour2)
	if cv2.contourArea(contour2) > area_threshold:

		image_gray_part = image_gray[y:y + h, x:x + w]

		ret2, image_threshold2 = cv2.threshold(image_gray_part, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)

		# detect lines_list
		lines, width, prec, nfa = lsd.detect(image_threshold2)
		if lines is not None:
			if lines.shape[0] < 70:
				cv2.drawContours(image_gray, [contour2], -1, 0, cv2.FILLED)
			else:
				# for line in lines:
				# 	# 获取线段的起点和终点坐标
				# 	x1, y1, x2, y2 = line[0].astype(int)
				# 	# 绘制线段
				# 	cv2.line(image_pydown_s, (x + x1, y + y1), (x + x2, y + y2), (0, 255, 0), 2)
				cv2.drawContours(image_pydown_s, [contour2], -1, (0, 255, 0, 1))
			lines_list2.append(lines)
	else:

		cv2.drawContours(image_gray, [contour2], -1, 0, cv2.FILLED)
plt.figure(dpi=300)
plt.xticks([])
plt.yticks([])
plt.imshow(cv2.cvtColor(image_pydown_s, cv2.COLOR_BGR2RGB))
plt.show()


In [8]:
import cv2
import numpy as np


def build_edge_preserving_pyramid(image, levels):
	# 创建高斯金字塔
	gaussian_pyramid = [image]
	for _ in range(levels - 1):
		image = cv2.pyrDown(image)
		gaussian_pyramid.append(image)

	# 创建拉普拉斯金字塔
	laplacian_pyramid = [gaussian_pyramid[levels - 1]]
	for i in range(levels - 2, -1, -1):
		expanded = cv2.pyrUp(gaussian_pyramid[i + 1], dstsize=gaussian_pyramid[i].shape[:2][::-1])
		laplacian = cv2.subtract(gaussian_pyramid[i], expanded)
		laplacian_pyramid.append(laplacian)

	return gaussian_pyramid, laplacian_pyramid


def reconstruct_image(gaussian_pyramid, laplacian_pyramid, levels, target_level):
	# 合并拉普拉斯金字塔和高斯金字塔
	reconstructed_image = cv2.add(gaussian_pyramid[target_level], laplacian_pyramid[levels - target_level - 1])
	return reconstructed_image


# 读取图像
image = cv2.imread(r"D:\Fenkx\Fenkx - General\AI\Dataset\BarCode\My Datasets\Factory\772FBFU0MPZZZS02H0_NG_BarCode_Camera3_1205171258.jpg")

# 设定金字塔级别
levels = 5

# 建立保留边缘信息的图像金字塔
gaussian_pyramid, laplacian_pyramid = build_edge_preserving_pyramid(image, levels)

# 指定层级，重建图像
target_level = 3
reconstructed_image = reconstruct_image(gaussian_pyramid, laplacian_pyramid, levels, target_level)

# 显示结果
cv2.namedWindow('Reconstructed Image', cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
cv2.imshow('Reconstructed Image', reconstructed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [14]:
# 创建高斯卷积核
sigma = 0.4
size = int(2 * np.ceil(3 * sigma) + 1)
kernel_gaussian = cv2.getGaussianKernel(size, sigma)

# 创建 LoG 卷积核
kernel_log = cv2.sepFilter2D(kernel_gaussian, -1, kernel_gaussian.T, kernel_gaussian)

print(kernel_log)


[[0.00327177]
 [0.07425046]
 [0.8482239 ]
 [0.07425046]
 [0.00327177]]
